In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
import tensorflow as tf

In [22]:
from tensorflow.examples.tutorials.mnist import input_data

In [23]:
mnist = input_data.read_data_sets("official/mnist/dataset.py", one_hot = True)

Extracting official/mnist/dataset.py/train-images-idx3-ubyte.gz
Extracting official/mnist/dataset.py/train-labels-idx1-ubyte.gz
Extracting official/mnist/dataset.py/t10k-images-idx3-ubyte.gz
Extracting official/mnist/dataset.py/t10k-labels-idx1-ubyte.gz


In [24]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [25]:
mnist.train.num_examples

55000

In [26]:
mnist.test.num_examples

10000

In [27]:
def init_weights(filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev = 0.1)
    return(tf.Variable(init_random_dist))

In [28]:
def init_bias(bias_shape):
    init_bias_vals = tf.constant(0.1, shape = bias_shape)
    return(tf.Variable(init_bias_vals))

In [29]:
def conv_layer_and_dot_product(inputs, filter_shape):
    filter_init_with_weights = init_weights(filter_shape)
    conv_layer_out = tf.nn.conv2d(inputs, filter_init_with_weights, strides = [1,1,1,1], padding = 'SAME')
    return(conv_layer_out)

In [30]:
def relu_layer_and_dot_product_plus_bias(inputs, filter_shape):
    bias = init_bias([filter_shape[3]])
    relu_layer_out = tf.nn.relu(inputs + bias)
    return(relu_layer_out)

In [31]:
def maxpool_2by2_and_reduce_spatial_size(inputs):
    pooling_layer_out = tf.nn.max_pool(inputs, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    return(pooling_layer_out)

In [32]:
def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs, output_size):
    in_size = int(inputs.get_shape()[1])
    wt = init_weights([in_size, output_size])
    bias = init_bias([output_size])
    fc_xW_plus_bias_outputs = tf.matmul(inputs, wt) + bias
    return(fc_xW_plus_bias_outputs)

In [33]:
x = tf.placeholder(tf.float32, shape = [None, 784])
y_true = tf.placeholder(tf.float32, [None,10])

In [34]:
x_image = tf.reshape(x,[-1, 28, 28, 1])

In [35]:
conv_layer_1_out \
    = conv_layer_and_dot_product(x_image, filter_shape=[5, 5, 1, 32])
conv_relu_layer_1_out \
    = relu_layer_and_dot_product_plus_bias(conv_layer_1_out, filter_shape=[5, 5, 1, 32])

In [36]:
pooling_layer_1_out = maxpool_2by2_and_reduce_spatial_size(conv_relu_layer_1_out)

In [37]:
conv_layer_2_out \
    = conv_layer_and_dot_product(pooling_layer_1_out, filter_shape=[5, 5, 32, 64])
conv_relu_layer_2_out \
    = relu_layer_and_dot_product_plus_bias(conv_layer_2_out, filter_shape=[5, 5, 32, 64])

In [38]:
pooling_layer_2_out = maxpool_2by2_and_reduce_spatial_size(conv_relu_layer_2_out)
pooling_layer_2_out_flat = tf.reshape(pooling_layer_2_out, [-1, 7*7*64])

In [39]:
fc_layer_1_out \
    = create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_out_flat, output_size=1024)
fc_relu_layer_1_out = tf.nn.relu(fc_layer_1_out)

In [40]:
hold_prob = tf.placeholder(tf.float32)
fc_dropout_out = tf.nn.dropout(fc_relu_layer_1_out, keep_prob = hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [41]:
y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_out,output_size=10)

In [42]:
softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred)
mean_cross_entropy = tf.reduce_mean(softmax_cross_entropy_loss)

In [43]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [44]:
cnn_trainer=optimizer.minimize(mean_cross_entropy)

In [45]:
vars_initializer = tf.global_variables_initializer()
steps=5000

In [46]:
with tf.Session() as sess:
    sess.run(vars_initializer)
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(cnn_trainer,feed_dict = {x:batch_x, y_true:batch_y,hold_prob:0.5})
        if i%100==0:
            print("Step :{}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            accuracy = tf.reduce_mean(tf.cast(matches,tf.float32))
            test_accuracy = sess.run(accuracy, feed_dict = {x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0})
            print(test_accuracy)
            print("\n")

Step :0
Accuracy: 
0.1131


Step :100
Accuracy: 
0.9482


Step :200
Accuracy: 
0.967


Step :300
Accuracy: 
0.9698


Step :400
Accuracy: 
0.9749


Step :500
Accuracy: 
0.9801


Step :600
Accuracy: 
0.9823


Step :700
Accuracy: 
0.9818


Step :800
Accuracy: 
0.9812


Step :900
Accuracy: 
0.9801


Step :1000
Accuracy: 
0.9832


Step :1100
Accuracy: 
0.9878


Step :1200
Accuracy: 
0.9837


Step :1300
Accuracy: 
0.9871


Step :1400
Accuracy: 
0.9879


Step :1500
Accuracy: 
0.9868


Step :1600
Accuracy: 
0.9902


Step :1700
Accuracy: 
0.9876


Step :1800
Accuracy: 
0.9872


Step :1900
Accuracy: 
0.9861


Step :2000
Accuracy: 
0.9859


Step :2100
Accuracy: 
0.9889


Step :2200
Accuracy: 
0.9888


Step :2300
Accuracy: 
0.9882


Step :2400
Accuracy: 
0.9905


Step :2500
Accuracy: 
0.99


Step :2600
Accuracy: 
0.9896


Step :2700
Accuracy: 
0.9864


Step :2800
Accuracy: 
0.989


Step :2900
Accuracy: 
0.9899


Step :3000
Accuracy: 
0.9882


Step :3100
Accuracy: 
0.9888


Step :3200
Accuracy: 
0.